In [2]:
from CNTools.datasets import load_ct_order, prep_t2d, Dataset
from CNTools.identification import CC, CFIDF, CNE, Spatial_LDA
from CNTools.smoothing import NaiveSmooth, HMRF
import os
import pandas as pd
import pickle

In [4]:
# Download CRC, T2D, and HLT data
# os.system('gdown --folder https://drive.google.com/drive/folders/1eC3ahSIzCZ5vq_bynSl3IX7V9w_bQgg8?usp=share_link')

In [5]:
df = prep_t2d(pd.read_csv('data/T2D/Cell-ID_by-islet.csv'))
ct_order = load_ct_order('data/T2D/ct_order.json')
ds = Dataset(df, ct_order)

In [6]:
# identifier = CC(n_cns=6, m=5)
identifier = CNE(n_cns=6, perp=15, lam=0.25)
# identifier = Spatial_LDA(n_cns=6, eps=100, b=0.25, train_size_fraction=0.989)

cns = identifier.fit(ds)

100%|██████████| 16/16 [00:01<00:00, 14.22it/s]


Entropy: 1.518, Size: 18.19


In [7]:
smoother = NaiveSmooth(ds=ds, n_cns=identifier.n_cns, feats=identifier.feats, s=3)
cns_smoothed_naive = smoother.fit(cns)

Entropy: 1.551, Size: 22.45


In [8]:
smoother = HMRF(ds=ds, n_cns=identifier.n_cns, eps=45, beta=9)
cns_smoothed_hmrf = smoother.fit(cns)

Entropy: 1.640, Size: 21.67


In [9]:
df.to_csv('data/T2D/T2D_df.csv', index=False)
pickle.dump(ds, open(f'data/T2D/T2D_ds.pkl', 'wb'))
pickle.dump(cns, open(f'cn/T2D/CNE/cns_n_cns=6_perp=15_lam=0.25.pkl', 'wb'))
pickle.dump(cns_smoothed_naive, open(f'cn/T2D/CNE/cns_n_cns=6_perp=15_lam=0.25_naive.pkl', 'wb'))
pickle.dump(cns_smoothed_hmrf, open(f'cn/T2D/CNE/cns_n_cns=6_perp=15_lam=0.25_hmrf.pkl', 'wb'))